In [ ]:
import os
from time import sleep
import dotenv
import pandas as pd
import base64
import json
from openai import OpenAI
from tqdm import tqdm

# Load environment variables
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Load book metadata
books = pd.read_csv("books/Aaltoes Library Books.csv")
books['book_image'] = "books/books_images/" + books['book_id'].astype(str) + ".jpeg"

# Dictionary to store the results
fetched_data = {}


In [48]:
counter = 0

# Loop over books (limiting to first 3 for testing)
for book_id, book_title, book_image in tqdm(books.values, desc="Processing books"):
    if not os.path.exists(book_image):
        continue

    try:
        counter += 1

        if counter % 10 == 0:
            sleep(60)
            print(f"Sleeping for 60 seconds. Counter: {counter}")
        # Read and encode image in base64
        with open(book_image, "rb") as image_file:
            image_data = image_file.read()
            encoded_image = base64.b64encode(image_data).decode()

        # Prompt message
        prompt_text = (
            f'Book image is sent as attached file. '
            f'Book title provided: "{book_title}". '
            f'Please return a JSON object with keys "author", "year", "title", and "description". '
            f'If some information is not available, use null for that field. '
            f'If you are aware of the book, add a short summary of the book in the "description" field. Keep it short, maximum 50 words.'
            f'Years is when the book was published for the first time. If you are not sure, use null.'

        )

        # GPT-4 Turbo with Vision call
        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_text},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
                    ]
                }
            ],
            temperature=0.0,
            max_tokens=1000,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "book_info",
                    "strict": True,
                    "schema": {
                        "type": "object",
                        "properties": {
                            "author": {"type": ["string", "null"]},
                            "year": {"type": ["string", "null"]},
                            "title": {"type": ["string", "null"]},
                            "description": {"type": ["string", "null"]}
                        },
                        "required": ["author", "title", "description", "year"],
                        "additionalProperties": False
                    }
                }
            }
        )

        try:
            content = response.choices[0].message.content
            fetched_data[book_id] = json.loads(content)
            print(fetched_data[book_id])
        except json.JSONDecodeError:
            fetched_data[book_id] = {"error": "Invalid JSON", "raw_response": content}

    except Exception as e:
        fetched_data[book_id] = {"error": str(e)}

# Save the results
with open("fetched_books_metadata.json", "w", encoding="utf-8") as f:
    json.dump(fetched_data, f, indent=2, ensure_ascii=False)


Processing books:   1%|          | 1/132 [00:08<18:01,  8.25s/it]

{'author': 'M. Scott Peck, M.D.', 'year': '1978', 'title': 'The Road Less Traveled', 'description': 'This book explores the importance of self-discipline, love, and spiritual growth, offering insights into personal development and the challenges of life.'}


Processing books:   2%|▏         | 2/132 [00:10<10:50,  5.00s/it]

{'author': 'Don Tapscott and Anthony D. Williams', 'year': '2006', 'title': 'Wikinomics: How Mass Collaboration Changes Everything', 'description': 'This book explores how mass collaboration through the internet transforms business, society, and the economy, highlighting the power of collective intelligence and the impact of digital technologies.'}


Processing books:   2%|▏         | 3/132 [00:13<08:41,  4.04s/it]

...

{'author': None, 'year': None, 'title': 'WELCOME TO FINNISH DESIGN THINKING', 'description': 'This book explores the principles and practices of Finnish design thinking, emphasizing creativity, innovation, and user-centered design. It serves as a guide for integrating these concepts into various fields.'}


Processing books:  92%|█████████▏| 122/132 [19:34<00:37,  3.78s/it]

{'author': None, 'year': None, 'title': 'welcome to finnish design thinking', 'description': 'This book explores the principles and practices of Finnish design thinking, emphasizing creativity, innovation, and user-centered design. It serves as a guide for integrating these concepts into various fields.'}
Sleeping for 60 seconds. Counter: 120


Processing books: 100%|██████████| 132/132 [20:37<00:00,  9.37s/it]

{'author': 'The Lunicorn', 'year': None, 'title': 'Startup Buzzword Dictionary', 'description': 'A humorous guide to the jargon and buzzwords commonly used in the startup world, helping readers navigate the often confusing language of entrepreneurship.'}


In [46]:
fetched_data

{1: {'author': 'M. Scott Peck, M.D.',
  'year': '1978',
  'title': 'The Road Less Traveled',
  'description': 'This book explores the importance of self-discipline, love, and spiritual growth, offering insights into personal development and the challenges of life.'}}

In [49]:
import json
import pandas as pd

def read_json_to_dataframe(file_path="fetched_books_metadata.json"):
    try:
        # Read the JSON file
        with open(file_path, "r", encoding="utf-8") as f:
            book_data = json.load(f)
        
        # Prepare data for DataFrame
        rows = []
        for book_id, book_info in book_data.items():
            if isinstance(book_info, dict) and "error" not in book_info:
                row = {
                    "book_id": book_id,
                    "author": book_info.get("author"),
                    "year": book_info.get("year"),
                    "title": book_info.get("title"),
                    "description": book_info.get("description")
                }
                rows.append(row)
        
        # Create DataFrame
        df = pd.DataFrame(rows)
        
        # Reorder columns
        df = df[["book_id", "author", "year", "title", "description"]]
        
        return df
        
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: File '{file_path}' is not a valid JSON file.")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [51]:
# Read JSON and create DataFrame
df = read_json_to_dataframe()
df.to_csv("books_dataframe.csv", index=False)
df

,book_id,author,year,title,description
0,1,"M. Scott Peck, M.D.",1978,The Road Less Traveled,This book explores the importance of self-disc...
1,2,Don Tapscott and Anthony D. Williams,2006,Wikinomics: How Mass Collaboration Changes Eve...,This book explores how mass collaboration thro...
2,3,Hemant Taneja with Kevin Maney,2021,Unscaled - How AI and a New Generation of Upst...,This book explores how AI and innovative start...
3,4,Petteri Kilpinen,None,Inspiroitunut,This book explores leadership and success in a...
4,5,Richard Carlson,None,älä hikeenny raha-asioista,This book offers practical advice on managing ...
...,...,...,...,...,...
111,119,None,None,welcome to finnish design thinking,This book explores the principles and practice...
112,120,None,None,welcome to finnish design thinking,This book explores the principles and practice...
113,121,None,None,WELCOME TO FINNISH DESIGN THINKING,This book explores the principles and practice...
114,122,None,None,welcome to finnish design thinking,This book explores the principles and practice...


In [4]:
import pandas as pd

In [7]:
df = pd.read_csv("books/books.csv")
df

,book_id,author,year,title,description,image
0,1,"M. Scott Peck, M.D.",1978.0,The Road Less Traveled,"The book delves into self-discipline, love, an...",1.jpeg
1,2,Don Tapscott and Anthony D. Williams,2006.0,Wikinomics: How Mass Collaboration Changes Eve...,It examines how mass collaboration through the...,2.jpeg
2,3,Hemant Taneja with Kevin Maney,2021.0,Unscaled - How AI and a New Generation of Upst...,The book discusses how AI-driven startups are ...,3.jpeg
3,4,Petteri Kilpinen,2017.0,Inspiroitunut,"The book focuses on leadership, motivation, an...",4.jpeg
4,5,Richard Carlson,2001.0,Älä hikeenny raha-asioista,The book provides practical advice on reducing...,5.jpeg
...,...,...,...,...,...,...
111,119,Edited by Pauliina Savolainen,2012.0,Welcome to Finnish Design Thinking,Delves into Finland's innovative design method...,119.jpeg
112,120,Edited by Pauliina Savolainen,2012.0,Welcome to Finnish Design Thinking,Delves into Finland's innovative design method...,120.jpeg
113,121,Edited by Pauliina Savolainen,2012.0,Welcome to Finnish Design Thinking,Delves into Finland's innovative design method...,121.jpeg
114,122,Edited by Pauliina Savolainen,2012.0,Welcome to Finnish Design Thinking,Delves into Finland's innovative design method...,122.jpeg


In [8]:
df['image'] = df['book_id'].astype(str) + ".jpg"

df

,book_id,author,year,title,description,image
0,1,"M. Scott Peck, M.D.",1978.0,The Road Less Traveled,"The book delves into self-discipline, love, an...",1.jpg
1,2,Don Tapscott and Anthony D. Williams,2006.0,Wikinomics: How Mass Collaboration Changes Eve...,It examines how mass collaboration through the...,2.jpg
2,3,Hemant Taneja with Kevin Maney,2021.0,Unscaled - How AI and a New Generation of Upst...,The book discusses how AI-driven startups are ...,3.jpg
3,4,Petteri Kilpinen,2017.0,Inspiroitunut,"The book focuses on leadership, motivation, an...",4.jpg
4,5,Richard Carlson,2001.0,Älä hikeenny raha-asioista,The book provides practical advice on reducing...,5.jpg
...,...,...,...,...,...,...
111,119,Edited by Pauliina Savolainen,2012.0,Welcome to Finnish Design Thinking,Delves into Finland's innovative design method...,119.jpg
112,120,Edited by Pauliina Savolainen,2012.0,Welcome to Finnish Design Thinking,Delves into Finland's innovative design method...,120.jpg
113,121,Edited by Pauliina Savolainen,2012.0,Welcome to Finnish Design Thinking,Delves into Finland's innovative design method...,121.jpg
114,122,Edited by Pauliina Savolainen,2012.0,Welcome to Finnish Design Thinking,Delves into Finland's innovative design method...,122.jpg


In [11]:
df['year'][df['year'].isna()==False] = df['year'][df['year'].isna()==False].astype(int).astype(str)
df.to_csv("books/books.csv", index=False)